# YOLOv7 Brain Tumor Segmentation Pipeline
This Colab notebook trains and evaluates YOLOv7 on brain tumor MRI images.

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Define dataset paths
base_path = '/content/drive/MyDrive/yolov7_tumor_segmentation'
train_path = f"{base_path}/train"
val_path = f"{base_path}/validation"
test_path = f"{base_path}/test"

# Verify dataset folders
for folder in [f"{train_path}/images", f"{train_path}/labels", f"{val_path}/images", f"{val_path}/labels"]:
    assert os.path.exists(folder), f"Missing folder: {folder}"

print("✅ Dataset structure verified")

In [ ]:
# Step 2: Clone YOLOv7 repository and install dependencies
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt

In [ ]:
# Step 3: Create data.yaml dynamically
data_yaml = f"""
train: {train_path}/images
val: {val_path}/images
test: {test_path}/images

nc: 1
names: ['tumor']
"""

with open('/content/yolov7/data_tumor.yaml', 'w') as f:
    f.write(data_yaml)

print("✅ data.yaml created")

In [ ]:
# Step 4: Train YOLOv7 model
!python train.py --img 640 --batch 16 --epochs 50 \
  --data /content/yolov7/data_tumor.yaml \
  --weights yolov7.pt --device 0 --name tumor_yolov7

In [ ]:
# Step 5: Run inference on test images
!python detect.py --weights runs/train/tumor_yolov7/weights/best.pt \
  --source {test_path}/images \
  --conf 0.25 --name tumor_inference